In [ ]:
from dotenv import dotenv_values
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner
from datasets import Dataset
from config import postprocessing
from log import enlayer1_3epochs_4bits__ft_params as models_params

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']

splits = postprocessing.splits
max_new_tokens_factor_list = postprocessing.max_new_tokens_factor_list
n_shots_inference_list = postprocessing.n_shots_inference_list
similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

# f"data/test_data_processed/en_nShots{n_shots_inference}_maxNewTokensFactor{max_new_tokens_factor}.csv"
eval_data = Dataset.from_csv('data/test_data_processed/postprocessor_output_no_max_new_tokens.csv') 

/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.env/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
output_cleaner = OutputCleaner()
cleaned_data = output_cleaner.apply_cleaning(eval_data)
evaluator = Evaluator(data=cleaned_data, offset=False)
results = evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal[0], similar_is_equal_threshold=similar_is_equal_threshold[0])
results['precision'], results['recall'], results['f1']

ORIGINAL model_response:  ['diagnosed', 'unveiled', 'referred', 'hypokalemia', 'hypotension', 'dyslipidemia', 'diabetes mellitus', 'hypertension', 'A 46-year-old man', '4-months', '1-month']
sto analizzando: " hypotension " e ho trovato:  ['hypertension']
sto analizzando: " diabetes mellitus " e ho trovato:  ['new-onset diabetes mellitus']
GROUND THRUTH:  ['hypertension', 'dyslipidemia', 'diagnosed', 'mellitus', 'referred', 'hypokalemia', 'new-onset diabetes mellitus', 'A 46-year-old man', '4-months', '1-month']
NEW model_response:  ['diagnosed', 'unveiled', 'referred', 'hypokalemia', 'hypertension', 'dyslipidemia', 'new-onset diabetes mellitus', 'hypertension', 'A 46-year-old man', '4-months', '1-month'] 


ORIGINAL model_response:  ['study', 'tests', 'indicated ECS']
sto analizzando: " indicated ECS " e ho trovato:  ['indicated']
GROUND THRUTH:  ['study', 'tests', 'indicated', 'ECS']
NEW model_response:  ['study', 'tests', 'indicated'] 


ORIGINAL model_response:  ['findings', 'point

(0.63486529318542, 0.5431127982646421, 0.5854157533245652)

In [ ]:

load_in_4bit_list = model_loading_params.load_in_4bit

bnb_4bit_quant_type_list = model_loading_params.bnb_4bit_quant_type
bnb_4bit_compute_dtype_list = model_loading_params.bnb_4bit_compute_dtype

llm_int8_threshold_list = model_loading_params.llm_int8_threshold

r_list = lora_params.r
lora_alpha_list = lora_params.lora_alpha
lora_dropout_list = lora_params.lora_dropout

gradient_accumulation_steps_list = training_params.gradient_accumulation_steps
learning_rate_list = training_params.learning_rate

for model_loading_params_idx in range(len(load_in_4bit_list)):
  
  load_in_4bit = load_in_4bit_list[model_loading_params_idx]
  load_in_8bit = not load_in_4bit
  bnb_4bit_quant_type = bnb_4bit_quant_type_list[model_loading_params_idx]
  bnb_4bit_compute_dtype = bnb_4bit_compute_dtype_list[model_loading_params_idx]
  llm_int8_threshold = llm_int8_threshold_list[model_loading_params_idx]
  
  for r in r_list:
    for lora_alpha in lora_alpha_list:
      for lora_dropout in lora_dropout_list:
        for gradient_accumulation_steps in gradient_accumulation_steps_list:
          for learning_rate in learning_rate_list:
            nbits = 4
            if load_in_8bit:
              nbits = 8
            ADAPTERS_CHECKPOINT = f"ferrazzipietro/{config.model_name}_adapters_{config.TRAIN_LAYER}_{nbits}_{bnb_4bit_compute_dtype}_{r}_{lora_alpha}_{lora_dropout}_{gradient_accumulation_steps}_{learning_rate}"
            main(ADAPTERS_CHECKPOINT,
                  load_in_4bit, bnb_4bit_quant_type, bnb_4bit_compute_dtype, llm_int8_threshold,
                  r, lora_alpha, lora_dropout,
                  gradient_accumulation_steps,learning_rate)
